## Sage Data Merge

dupes:
    - yQ7pYy
    - ksJM3Y
    

In [2]:
import os, pandas as pd, numpy as np, re
from functools import reduce
pd.set_option('display.max_columns', 100)

#NOTE TO SELF: redo the relative paths.
sage = '/home/wraikes/Dropbox/partnership/DMTdata_8_31_17/sage'
os.chdir(sage)

In [3]:
#GLOBAL VARIABLE
new_dfs = {}

#Functions for data processing
def remove_dupes(df):
    
    test_users = [
        'ThpMV2Achc', 
        'SEkQVTCe6j', 
        'Wh8NSX3DHL', 
        'SaXFr2kPZa', 
        'VWUcSp4TeH', 
        'yXEfAmW682', 
        'gwEpQR8j9B',
        'WbbNWM4RAF', 
        'D5bzYrfd8E', 
        'LJcmEFWp74', 
        'ULoF3MM1nN'
    ]

    diff_study = [
        'rL8eA3',
        'rLg5xs',
        'rLrD9h',
        'rLP7H2',
        'rL6s6h',
        'aOyzBg',
        'aORA43',
        'aOh48U',
        'aOLu4K',
        'aOQtxv',
        'aO5TvQ',
        'mPC9S8',
        'mPgquX',
        'mP5xkB',
        'mPSQvh',
        'mPYk2p',
        'mP3rbd'
    ]

    
    return df[~df.externalId.isin(test_users + diff_study)]

def dupe_check(df):
    return len(df.externalId) == len(df.externalId.unique()) and len(df.externalId) > 0

def new_cols(df, append):
    # Remove: 'metadata.json.' and 'data.json.'.
    col_re = re.compile('metadata.json.|data.json.')
    df.columns = list(map(lambda x: re.sub(col_re, '', x), df.columns))
    
    # Append df identifier to columns.
    df.columns = list(map(lambda x: append + x, df.columns))
    df = df.rename(columns = {append + 'externalId': 'externalId'})

    return df

def create_new_df(df, att, var_1, var_2=None):
    cols = df.columns
    
    new_df = pd.DataFrame(columns = cols)
        
    for ix, row in df.iterrows():
        if var_2:
            if row[var_1] == att[1] and row[var_2] == att[2]:
                new_df = new_df.append(row, ignore_index=True)
        else:
            if row[var_1] == att[1]:
                new_df = new_df.append(row, ignore_index=True)
    
    return new_df

def df_merge(df, attributes, var_1, var_2=None):
    dfs = []
    
    for att in attributes:
        new_df = create_new_df(df, att, var_1, var_2)
        new_df = new_cols(new_df, att[0])
        dfs.append(new_df)
    
    df_merge = reduce(lambda left, right: pd.merge(left, 
                                                   right, 
                                                   how = 'outer',
                                                   on='externalId'),
                      dfs)
    
    return df_merge

def final_check(df, name):
    if dupe_check(df):
        new_dfs[name] = df
        print('Done!')
    else:
        print('False')

### Process files that do not need cleaning / restructuring.

In [6]:
files_to_exclude = ['digital-marshmallow-status_8.31.17.csv',
                    'digital-marshmallow-appVersion_8.31.17.csv']

files_as_is = []
files_to_modify = []

for file in os.listdir():
    if file not in files_to_exclude:
        df = pd.read_csv(file)
        df = remove_dupes(df)
        if dupe_check(df):
            files_as_is.append(file)
        else:
            files_to_modify.append(file)

files_as_is.sort()

In [7]:
len(files_as_is)

8

In [9]:
files_to_modify

['digital-marshmallow-morning_notification_time-v3_8.31.17.csv',
 'digital-marshmallow-generally_sem_diff_bl-v2_8.31.17.csv',
 'digital-marshmallow-bart-v4_8.31.17.csv',
 'digital-marshmallow-goNoGo-v2_8.31.17.csv',
 'digital-marshmallow-morning_yesterday_sem_diff-v2_8.31.17.csv',
 'digital-marshmallow-delay_discounting_raw-v6_8.31.17.csv',
 'digital-marshmallow-evening_sem_diff-v2_8.31.17.csv',
 'digital-marshmallow-comments-v2_8.31.17.csv',
 'digital-marshmallow-discounting_raw-v2_8.31.17.csv',
 'digital-marshmallow-morning_sem_diff-v2_8.31.17.csv',
 'digital-marshmallow-pam-v2_8.31.17.csv',
 'digital-marshmallow-morning_behavior_choices-v1_8.31.17.csv',
 'digital-marshmallow-pam_multiple-v2_8.31.17.csv',
 'digital-marshmallow-behavior_choices_4_bl-v2_8.31.17.csv',
 'digital-marshmallow-evening_notification_time-v2_8.31.17.csv',
 'digital-marshmallow-morning_yesterday_likert-v2_8.31.17.csv',
 'digital-marshmallow-demographics-v2_8.31.17.csv']

In [11]:
files_as_is

['digital-marshmallow-as_a_child_likert_bl-v2_8.31.17.csv',
 'digital-marshmallow-behavior_choices_1_bl-v1_8.31.17.csv',
 'digital-marshmallow-behavior_likert_21-v2_8.31.17.csv',
 'digital-marshmallow-behavior_likert_bl-v2_8.31.17.csv',
 'digital-marshmallow-comments_21-v2_8.31.17.csv',
 'digital-marshmallow-generally_sem_diff_21-v2_8.31.17.csv',
 'digital-marshmallow-past_year_likert_21-v2_8.31.17.csv',
 'digital-marshmallow-past_year_likert_bl-v3_8.31.17.csv']

In [12]:
# CHECK TO MAKE SURE FILE NAMES SYNC UP WITH files_as_is
names_as_is = [
    'SAGE_as_a_child_-_',
    'SAGE_behavior_choices_1_-_',
    'SAGE_behavior_lk_21_-_',
    'SAGE_behavior_lk_bl_-_',
    'SAGE_comments_21_-_',
    'SAGE_generally_sem_21_-_',
    'SAGE_past_year_likert_21_-_',
    'SAGe_past_year_likert_bl_-_'
]

# names_as_is = [
#     'SAGE_generally_sem_bl_-_',
#     'SAGe_past_year_likert_bl_-_',
#     'SAGE_as_a_child_-_',
#     'SAGE_behavior_lk_21_-_',
#     'SAGE_past_year_likert_21_-_',
#     'SAGE_comments_21_-_',
#     'SAGE_generally_sem_21_-_',
#     'SAGE_behavior_lk_bl_-_',
#     'SAGE_behavior_choices_1_-_',
#     'SAGE_demos_-_'
# ]

In [13]:
for file, name in zip(files_as_is, names_as_is):
    df = pd.read_csv(file)
    df = remove_dupes(df)
    df = new_cols(df, name)
    new_dfs[name] = df

In [14]:
all([dupe_check(df) for name, df in new_dfs.items()])

True

### Dupe: yQ7pYy

In [15]:
demos = pd.read_csv('digital-marshmallow-demographics-v2_8.31.17.csv')
demos = remove_dupes(demos)

dupe_check(demos)

False

In [16]:
demos.externalId[demos.externalId.duplicated()]

91    yQ7pYy
Name: externalId, dtype: object

In [17]:
gen = pd.read_csv('digital-marshmallow-generally_sem_diff_bl-v2_8.31.17.csv')
gen = remove_dupes(gen)

dupe_check(gen)

False

In [18]:
gen.externalId[gen.externalId.duplicated()]

95    yQ7pYy
Name: externalId, dtype: object

In [19]:
gen[gen.externalId == 'yQ7pYy']

,ROW_ID,ROW_VERSION,recordId,appVersion,phoneInfo,uploadDate,healthCode,externalId,dataGroups,createdOn,createdOnTimeZone,userSharingScope,metadata.json.UUID,metadata.json.taskIdentifier,metadata.json.taskRunUUID,metadata.json.startDate,metadata.json.startDate.timezone,metadata.json.endDate,metadata.json.endDate.timezone,metadata.json.groupLabel,data.json.confusion_bl,data.json.focus_bl,data.json.organized_bl,data.json.intentional_bl,data.json.bored_bl,data.json.proud_bl,data.json.efficient_bl,data.json.energetic_bl,data.json.calm_bl,data.json.conservative_bl,data.json.frustrated_bl,data.json.cautious_bl,data.json.sad_bl,data.json.lonely_bl,data.json.optimistic_bl,data.json.determined_bl,data.json.stressed_bl,data.json.positive_bl,data.json.healthy_bl,data.json.productive_bl
85,85,39,485bed95-7394-432f-8ba5-8bda3992136a,"version 1.2, build 18",iPhone 5S; iOS/10.3.3,8/4/2017,2bb53936-ef7c-46ae-b6ee-844ed7b9d325,yQ7pYy,NaN,1.501770e+12,-400,ALL_QUALIFIED_RESEARCHERS,7F670D2A-2F84-4F8E-938A-7C2422ABE258,baseline,B6FA8753-F80C-4F02-AC71-624C33D5F13F,NaN,NaN,NaN,NaN,86,1,7,2,7,8,2,2,2,5,9,9,2,9,9,1,1,5,1,3,2
95,95,43,bf185f0a-024e-42bf-8866-f5e22cbb5455,"version 1.2, build 18",iPhone 5S; iOS/10.3.3,8/26/2017,2bb53936-ef7c-46ae-b6ee-844ed7b9d325,yQ7pYy,NaN,1.501770e+12,-400,ALL_QUALIFIED_RESEARCHERS,7F670D2A-2F84-4F8E-938A-7C2422ABE258,baseline,B6FA8753-F80C-4F02-AC71-624C33D5F13F,NaN,NaN,NaN,NaN,86,1,7,2,7,8,2,2,2,5,9,9,2,9,9,1,1,5,1,3,2


In [22]:
demos[demos.externalId == 'yQ7pYy']

,ROW_ID,ROW_VERSION,recordId,appVersion,phoneInfo,uploadDate,healthCode,externalId,dataGroups,createdOn,createdOnTimeZone,userSharingScope,metadata.json.UUID,metadata.json.taskIdentifier,metadata.json.taskRunUUID,metadata.json.startDate,metadata.json.startDate.timezone,metadata.json.endDate,metadata.json.endDate.timezone,metadata.json.groupLabel,data.json.gender,data.json.age,data.json.zip_code,data.json.education,data.json.employment,data.json.ethnicity,data.json.religion
85,85,38,9b9508c1-07e3-40df-9a55-95eaa924d697,"version 1.2, build 18",iPhone 5S; iOS/10.3.3,8/4/2017,2bb53936-ef7c-46ae-b6ee-844ed7b9d325,yQ7pYy,NaN,1.501770e+12,-400,ALL_QUALIFIED_RESEARCHERS,CB78206A-2DE0-40A0-A580-A3ADD682F4FD,baseline,B6FA8753-F80C-4F02-AC71-624C33D5F13F,1.501770e+12,-400,1.501770e+12,-400,86,female,44,11783,Graduate degree,NaN,White,"[""Religious (no services)""]"
91,91,41,3cdf3f5f-2b94-4bca-ad31-4e3c8e3c5086,"version 1.2, build 18",iPhone 5S; iOS/10.3.3,8/22/2017,2bb53936-ef7c-46ae-b6ee-844ed7b9d325,yQ7pYy,NaN,1.501770e+12,-400,ALL_QUALIFIED_RESEARCHERS,CB78206A-2DE0-40A0-A580-A3ADD682F4FD,baseline,B6FA8753-F80C-4F02-AC71-624C33D5F13F,1.501770e+12,-400,1.501770e+12,-400,86,female,44,11783,Graduate degree,NaN,White,"[""Religious (no services)""]"


### DataFrame: Bart_V4

In [9]:
bart_v4 = pd.read_csv('digital-marshmallow-bart-v4_8.31.17.csv')
bart_v4 = remove_dupes(bart_v4)

dupe_check(bart_v4)

False

In [10]:
bart_attributes = [
    ['SAGE_bart_bl_0.25_-_', 'baseline', 'BART0.25'],
    ['SAGE_bart_bl_250_-_', 'baseline', 'BART250.00'],
    ['SAGE_bart_21_0.25_-_', '21-day-assessment', 'BART0.25'],
    ['SAGE_bart_21_250_-_', '21-day-assessment', 'BART250.00']
]

In [11]:
bart_v4 = df_merge(bart_v4, 
                   bart_attributes,
                   var_1='metadata.json.taskIdentifier',
                   var_2='data.json.variable_label')

In [12]:
final_check(bart_v4, 'bart_v4')

Done!


### DataFrame: Behavior_choices_4

In [13]:
behavior_4 = pd.read_csv('digital-marshmallow-behavior_choices_4_bl-v2_8.31.17.csv')
behavior_4 = remove_dupes(behavior_4)

dupe_check(behavior_4)

False

In [14]:
behavior_4_attributes = [
    ['SAGE_behavior_4_bl_-_', 'baseline'],
]

In [15]:
behavior_4 = df_merge(behavior_4, 
                      behavior_4_attributes,
                      var_1='metadata.json.taskIdentifier')

In [16]:
final_check(behavior_4, 'behave_4')

Done!


### DataFrame: Delay Discounting

In [17]:
delay = pd.read_csv('digital-marshmallow-delay_discounting_raw-v6_8.31.17.csv')
delay = remove_dupes(delay)

dupe_check(delay)

False

In [18]:
bl = 'baseline'
_21 = '21-day-assessment'

delay_attributes = [
    ['SAGE_delay_bl_time_6_month_-_', bl, 'dd_time_6_month'],
    ['SAGE_delay_bl_money_6_month_-_', bl, 'dd_money_6_month'],
    ['SAGE_delay_bl_money_1_month_-_', bl, 'dd_money_1_month'],
    ['SAGE_delay_bl_time_1_year_-_', bl, 'dd_time_1_year'],
    ['SAGE_delay_21_time_6_month_-_', _21, 'dd_time_6_month'],
    ['SAGE_delay_21_money_6_month_-_', _21, 'dd_money_6_month'],
    ['SAGE_delay_21_money_1_month_-_', _21, 'dd_money_1_month'],
    ['SAGE_delay_21_time_1_year_-_', _21, 'dd_time_1_year']
]

In [19]:
delay = df_merge(delay, 
                 delay_attributes, 
                 var_1='metadata.json.taskIdentifier',
                 var_2='data.json.variableLabel')

In [20]:
final_check(delay, 'delay')

Done!


### DataFrame: Discounting Raw

In [21]:
discount = pd.read_csv('digital-marshmallow-discounting_raw-v2_8.31.17.csv')
discount = remove_dupes(discount)

dupe_check(discount)

False

In [22]:
bl = 'baseline'
_21 = '21-day-assessment'

discount_attributes = [
    ['SAGE_discount_bl_money_-_', bl, 'pd_constant_money'],
    ['SAGE_discount_bl_prob_-_', bl, 'pd_constant_probabiliy'],
    ['SAGE_discount_21_money_-_', _21, 'pd_constant_money'],
    ['SAGE_discount_21_prob_-_', _21, 'pd_constant_probability']
]

discount = df_merge(discount, 
                    discount_attributes,
                    var_1='metadata.json.taskIdentifier',
                    var_2='data.json.variableLabel')

In [23]:
final_check(discount, 'discount')

Done!


### DataFrame: Evening Notification

In [24]:
evening_note = pd.read_csv('digital-marshmallow-evening_notification_time-v2_8.31.17.csv')
evening_note = remove_dupes(evening_note)

dupe_check(evening_note)

False

In [25]:
evening_note_attributes = [
    ['SAGE_evening_note_bl_-_', 'baseline']
]

evening_note_bl = df_merge(evening_note, 
                           evening_note_attributes,
                           var_1='metadata.json.taskIdentifier')

In [26]:
final_check(evening_note_bl, 'evening_note')

Done!


### DataFrame: GoNoGo - PLACEHOLDER (Extra Record - ksJM3Y)

In [27]:
gonogo = pd.read_csv('digital-marshmallow-goNoGo-v2_8.31.17.csv')
gonogo = remove_dupes(gonogo)

dupe_check(gonogo)

False

In [28]:
bl = 'baseline'
_21 = '21-day-assessment'

gonogo_attributes = [
    ['SAGE_gonogo_bl_stable_-_', bl, 'go_no_go_stable_stimulus_active_task'],
    ['SAGE_gonogo_21_variable_-_', bl, 'go_no_go_variable_stimulus_active_task'],
    ['SAGE_gonogo_bl_stable_-_', _21, 'go_no_go_stable_stimulus_active_task'],
    ['SAGE_gonogo_21_variable_-_', _21, 'go_no_go_variable_stimulus_active_task']
]

gonogo = df_merge(gonogo, 
                  gonogo_attributes,
                  var_1='metadata.json.taskIdentifier',
                  var_2='data.json.variable_label')

In [29]:
final_check(gonogo, 'gonogo')

False


In [30]:
gonogo.externalId[gonogo.externalId.duplicated()]

13    ksJM3Y
Name: externalId, dtype: object

### DataFrame: Morning Notifications

In [31]:
morning_note = pd.read_csv('digital-marshmallow-morning_notification_time-v3_8.31.17.csv')
morning_note = remove_dupes(morning_note)

dupe_check(morning_note)

False

In [32]:
morning_note_attributes = [
    ['SAGE_morning_note_bl_-_', 'baseline']
]

morning_note = df_merge(morning_note, 
                        morning_note_attributes,
                        var_1='metadata.json.taskIdentifier')

In [33]:
final_check(morning_note, 'morning_note')

Done!


### DataFrame: PAM Multiple

In [34]:
pam_mult = pd.read_csv('digital-marshmallow-pam_multiple-v2_8.31.17.csv')
pam_mult = remove_dupes(pam_mult)

dupe_check(pam_mult)

False

In [35]:
pam_mult_attributes = [
    ['SAGE_pam_mult_bl_-_', 'baseline'],
    ['SAGE_pam_mult_21_-_', '21-day-assessment']
]

pam_mult = df_merge(pam_mult, 
                    pam_mult_attributes,
                    var_1='metadata.json.taskIdentifier')

In [36]:
final_check(pam_mult, 'pam_mult')

Done!


### Final Merge of All Sage Data

In [37]:
final_df = reduce(lambda left, right: pd.merge(left, right, how = 'outer',
                                               on='externalId'), 
                  new_dfs.values())

In [38]:
dupe_check(final_df)

True

In [39]:
os.chdir('/home/wraikes/Programming/Partnership/dmt/merged_data/')
final_df.to_csv('FINAL_SAGE.csv')